In [1]:
import pandas as pd
import numpy as np
from some_functions import get_db

In [2]:
readers = pd.read_csv(r"C:\Users\a814811\OneDrive - Atos\RecommenderSystem\readers.csv")
readers = readers.rename(columns={"id":"user_id", "art_id":"nzz_id"})
art_db = get_db(r'C:\Users\a814811\OneDrive - Atos\RecommenderSystem\art_clean_wt_all_popularity.csv')
art_db = art_db.loc[:,['nzz_id','author','department','popularity']] #skrócenie do potrzebnych rzeczy

In [3]:
read_counts = readers["user_id"].value_counts(sort=True)
read_counts = read_counts.rename_axis("user_id").reset_index(name="read_count")

# Biorę pod uwagę tylko użytkowników, którzy przeczytali minimum 5 artykułów
min_read_count = 3
read_counts = read_counts[read_counts["read_count"] > min_read_count]

readers = readers[readers["user_id"].isin(read_counts["user_id"])]

In [4]:
# Train/Test split
from sklearn.model_selection import train_test_split

random_state = None
readers_train, readers_test = train_test_split(readers,
                                   stratify=readers["user_id"], 
                                   test_size=0.20,
                                   random_state=random_state)
                                   
print(f"Train set size {len(readers_train)}")
print(f"test set size {len(readers_test)}")
unique_train = readers_train["nzz_id"].unique()
unique_test = readers_test["nzz_id"].unique()
print(f"n users in test {len(unique_test)}")
print(f"n users in train {len(unique_train)}")

Train set size 22284
test set size 5571
n users in test 3992
n users in train 10018


In [5]:

from popularity_model import *
from model_evaluator import ModelEvaluator
model_evaluator = ModelEvaluator(k_list = [5, 10, 15])

### sprawdzam submodele i główny
#### dla authora są mocno zaniżone wyniki, bo często nie rekomenduje nic a test tego nie uwzględnia, podobnie dla department (w mniejszym stopniu)
#### merged i popularity działają normalnie (zawsze coś jest rekomednowane)

In [6]:
# popularity model
p_model = Popularity_model(art_db,readers)
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(p_model, readers, readers_train, readers_test)
print('\nGlobal metrics:\n%s' % cf_global_metrics)

999 users processed

Global metrics:
{'modelName': 'p_model', 'recall@5': 0.23082341269841272, 'precision@5': 0.04616468253968254, 'f1_score@5': 0.07694113756613757, 'ndcg@5': 0.15875321247124569, 'recall@10': 0.3567448412698413, 'precision@10': 0.035674484126984124, 'f1_score@10': 0.06486269841269843, 'ndcg@10': 0.1998560374447315, 'recall@15': 0.44260039682539687, 'precision@15': 0.029506693121693118, 'f1_score@15': 0.05532504960317461, 'ndcg@15': 0.2217301724369588}


In [13]:
# author popularity model
p_model = Popularity_model_author(art_db,readers)
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(p_model, readers, readers_train, readers_test)
print(f'\nGlobal metrics:\n{cf_global_metrics}')

999 users processed

Global metrics:
{'modelName': 'p_model', 'recall@5': 0.11160119047619046, 'precision@5': 0.022320238095238098, 'f1_score@5': 0.037200396825396824, 'ndcg@5': 0.09526888143660713, 'recall@10': 0.11216626984126984, 'precision@10': 0.011216626984126985, 'f1_score@10': 0.020393867243867242, 'ndcg@10': 0.09566072567703654, 'recall@15': 0.11216626984126984, 'precision@15': 0.0074777513227513226, 'f1_score@15': 0.01402078373015873, 'ndcg@15': 0.095492105181566}


In [6]:
# department popularity model
p_model = Popularity_model_department(art_db,readers)
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(p_model, readers, readers_train, readers_test)
print(f'\nGlobal metrics:\n{cf_global_metrics}')

KeyboardInterrupt: 

In [19]:
# merged model
p_model = Popularity_model_merge(art_db,readers)
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(p_model, readers, readers_train, readers_test)
print(f'\nGlobal metrics:\n{cf_global_metrics}')

999 users processed

Global metrics:
{'modelName': 'p_model', 'recall@5': 0.23119087301587302, 'precision@5': 0.0462381746031746, 'f1_score@5': 0.07706362433862433, 'ndcg@5': 0.159438261869886, 'recall@10': 0.3545801587301587, 'precision@10': 0.03545801587301588, 'f1_score@10': 0.06446911976911977, 'ndcg@10': 0.19879871336548444, 'recall@15': 0.4402527777777777, 'precision@15': 0.029350185185185184, 'f1_score@15': 0.05503159722222222, 'ndcg@15': 0.22320654222567332}


In [20]:
cf_detailed_results_df['model'] = ['merged']*1000
cf_detailed_results_df.to_csv("test_results2", encoding="utf-8", index=False)

## Wizualizacja

In [15]:
from matplotlib import pyplot as plt